In [1]:
from dotenv import load_dotenv
import inspect
import os
import pandas as pd

from utils import init_flipside, execute_query

from queries import (
    bridge,
    dex,
    farcaster,
    lending,
    marketplace,
    nft,
    rwa,
    staking
)

In [2]:
load_dotenv()
FLIPSIDE_API_KEY = os.getenv('FLIPSIDE_API_KEY')
flipside = init_flipside(FLIPSIDE_API_KEY)

In [3]:
PRIMARY_KEY = 'uuid'
GROUPER_KEY = 'recipient'

df_apps = pd.read_csv('data/applications_reviewed.csv', index_col=0)

In [7]:
creator_addresses = []
onchain_apps = []
warpcast_apps = []

for _,app in df_apps.iterrows():
        
    app_data = {
        PRIMARY_KEY: app[PRIMARY_KEY],
        GROUPER_KEY: app[GROUPER_KEY],
        'project_type': app['project_type'],
        'category': app['category']
    }

    if app['has_flag']:
        continue
    
    if pd.isna(app['address']):
        app_data.update({'website': app['metadata_website']})
        warpcast_apps.append(app_data)
        continue

    if len(app['address']) != 42:
        continue

    app_data.update({
        'address': app['address'],
        'chain': app['chain']
    })
    
    if app['chain'] != 'All Superchain':
        onchain_apps.append(app_data)
        continue

    creator_addresses.append(app['address'])

    for chain in ['Base', 'Optimism', 'Zora']:
        temp_app_data = app_data.copy()
        temp_app_data.update({'chain': chain})
        onchain_apps.append(temp_app_data)
            
df_onchain = pd.DataFrame(onchain_apps)
df_warpcast = pd.DataFrame(warpcast_apps)
len(creator_addresses)

298

In [8]:
METRIC_MODULES = {
    'Art NFTs': {
        "module": nft,
        "params": ["chain", "address"]
    },
    'Other Media NFTs': {
        "module": nft,
        "params": ["chain", "address"]
    },
    'Bridges': {
        "module": bridge,
        "params": ["chain", "address"]
    },
    'DEX': {
        "module": dex,
        "params": ["chain", "address"]
    },
    'Channels': {
        "module": farcaster,
        "params": ["warpcast_url"]
    },
    'Frames': {
        "module": None, # needs to be updated
        "params": ["warpcast_url"]
    },
    'Lending': {
        "module": lending,
        "params": ["chain", "address"]
    },
    'Art Marketplace': {
        "module": marketplace,
        "params": ["chain", "address"]
    },
    'Other Media Marketplace': {
        "module": marketplace,
        "params": ["chain", "address"]
    },
    'Real World Assets': {
        "module": rwa,
        "params": ["chain", "address"]
    },
    'Staking': {
        "module": staking,
        "params": ["chain", "address"]
    },
    'Community & Curation': {
        "module": None,
        "params": ["chain", "address"]
    },
    'Gaming': {
        "module": None,
        "params": ["chain", "address"]
    },
    'Airdrop': {
        "module": None,
        "params": ["chain", "address"]
    },
    'Meme Community': {
        "module": None,
        "params": ["chain", "address"]
    },
    'Payments': {
        "module": None,
        "params": ["chain", "address"]
    },    
    'Donations': {
        "module": None,
        "params": ["chain", "address"]
    },
    'Identity': {
        "module": None,
        "params": ["chain", "address"]
    },
    'Betting & Prediction Markets': {
        "module": None,
        "params": ["chain", "address"]
    },
}

In [9]:
def query_project_metrics(project_data):    
    
    category = project_data['category']
    module_name = METRIC_MODULES.get(category).get('module')
    if not module_name:
        return
    
    metric_modules = inspect.getmembers(module_name, inspect.isfunction)
    expected_params = METRIC_MODULES.get(category).get('params', [])
    params = [project_data.get(p) for p in expected_params]
    if len(params) != len(expected_params):
        print("Unable to find params:", expected_params)
        return
    
    metrics = []    
    for (query_name, query_fn) in metric_modules:
        sql = query_fn(*params)
        value = execute_query(flipside, sql, project_data)
        metrics.append({
            **project_data,
            'query_name': query_name,
            'value': value
        })
    
    return metrics

In [10]:
df_onchain['chain'].value_counts()

chain
Base          415
Zora          302
Optimism      298
OP Mainnet     52
Mint            8
Mode            6
SwanChain       4
Kroma           4
Cyber           2
Orderly         2
Lisk            1
Redstone        1
Polynomial      1
Name: count, dtype: int64

In [11]:
nft_project = df_onchain[df_onchain['category'] == 'Art NFTs'].iloc[3]
nft_project
pm = query_project_metrics(nft_project)
pm

2024-09-19 17:35:36,460 - INFO - Query Attempt: uuid: 8c12c1a5-d565-4851-abf2-1e22724816b0, recipient: 0x72bA95D9701B0aE496AB4320010a43a764905b1b, project_type: Creator, category: Art NFTs, address: 0x72ba95d9701b0ae496ab4320010a43a764905b1b, chain: Base, Status: success, Execution Time: 12.47 seconds, Value: 0
2024-09-19 17:35:43,450 - INFO - Query Attempt: uuid: 8c12c1a5-d565-4851-abf2-1e22724816b0, recipient: 0x72bA95D9701B0aE496AB4320010a43a764905b1b, project_type: Creator, category: Art NFTs, address: 0x72ba95d9701b0ae496ab4320010a43a764905b1b, chain: Base, Status: success, Execution Time: 6.99 seconds, Value: None


[{'uuid': '8c12c1a5-d565-4851-abf2-1e22724816b0',
  'recipient': '0x72bA95D9701B0aE496AB4320010a43a764905b1b',
  'project_type': 'Creator',
  'category': 'Art NFTs',
  'address': '0x72ba95d9701b0ae496ab4320010a43a764905b1b',
  'chain': 'Base',
  'query_name': 'artnft_nam',
  'value': 0},
 {'uuid': '8c12c1a5-d565-4851-abf2-1e22724816b0',
  'recipient': '0x72bA95D9701B0aE496AB4320010a43a764905b1b',
  'project_type': 'Creator',
  'category': 'Art NFTs',
  'address': '0x72ba95d9701b0ae496ab4320010a43a764905b1b',
  'chain': 'Base',
  'query_name': 'creator_contracts',
  'value': None}]

In [12]:
pm

[{'uuid': '8c12c1a5-d565-4851-abf2-1e22724816b0',
  'recipient': '0x72bA95D9701B0aE496AB4320010a43a764905b1b',
  'project_type': 'Creator',
  'category': 'Art NFTs',
  'address': '0x72ba95d9701b0ae496ab4320010a43a764905b1b',
  'chain': 'Base',
  'query_name': 'artnft_nam',
  'value': 0},
 {'uuid': '8c12c1a5-d565-4851-abf2-1e22724816b0',
  'recipient': '0x72bA95D9701B0aE496AB4320010a43a764905b1b',
  'project_type': 'Creator',
  'category': 'Art NFTs',
  'address': '0x72ba95d9701b0ae496ab4320010a43a764905b1b',
  'chain': 'Base',
  'query_name': 'creator_contracts',
  'value': None}]

In [14]:
creator_apps = apps[
    (apps['has_flag']==0)
    & (apps['category'].isin(['Art NFTs', 'Other Media NFTs']))
]


In [6]:
results = query_project_metrics(apps.iloc[0])
results

2024-09-16 10:09:15,248 - INFO - Query Attempt: id: 0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab, category: DEX, chain: Base, address: 0xdef1c0ded9bec7f1a1670819833240f027b25eff, Status: success, Execution Time: 83.03 seconds, Value: 1646413
2024-09-16 10:09:27,208 - INFO - Query Attempt: id: 0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab, category: DEX, chain: Base, address: 0xdef1c0ded9bec7f1a1670819833240f027b25eff, Status: success, Execution Time: 11.96 seconds, Value: 0
2024-09-16 10:09:35,124 - ERROR - Query execution failed: QUERY_RUN_EXECUTION_ERROR: an error has occured while executing your query. errorName=OperationFailedError, errorMessage=SQL compilation error:
syntax error line 13 at position 39 unexpected '60'.
syntax error line 13 at position 46 unexpected 'PRECEDING'., errorData={'code': '001003', 'data': {'age': 0, 'pos': -1, 'line': -1, 'type': 'COMPILATION', 'queryId': '01b71211-0410-eae1-3d4f-83026d7485b3', 'sqlState': '4200

[{'id': '0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab',
  'category': 'DEX',
  'chain': 'Base',
  'address': '0xdef1c0ded9bec7f1a1670819833240f027b25eff',
  'query_name': 'dex_ntrxns',
  'value': 1646413},
 {'id': '0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab',
  'category': 'DEX',
  'chain': 'Base',
  'address': '0xdef1c0ded9bec7f1a1670819833240f027b25eff',
  'query_name': 'dex_num_tokens',
  'value': 0},
 {'id': '0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab',
  'category': 'DEX',
  'chain': 'Base',
  'address': '0xdef1c0ded9bec7f1a1670819833240f027b25eff',
  'query_name': 'dex_nusers',
  'value': None},
 {'id': '0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406961eb85a8bba931d7dab',
  'category': 'DEX',
  'chain': 'Base',
  'address': '0xdef1c0ded9bec7f1a1670819833240f027b25eff',
  'query_name': 'dex_sumtrxns',
  'value': 22188.398141304}]

In [28]:
pd.DataFrame(creator_addresses).to_csv("creator_addresses.csv")

In [29]:
",".join(creator_addresses)

'0xd0966b2d8d48e057105afa7a1b84bf1188fb00f0,0x72ba95d9701b0ae496ab4320010a43a764905b1b,0x5de2b5b72a0b49f7d4490a1fef61afa8415971f5,0xb1abf1039323f248514594a5ed1aef4f4fc95823,0x4b4324bcc6db9380abbbbd20b24a16c11fb5b38a,0x6826ba168776251688d4e3a6c4ac430cf4f185a8,0x2ea49c863be976ee9c10cb8ef382757e48f0b9e5,0x8f87d1feefa305a453174982df870fa93ebe2175,0x21339e2ce5ce1d7a19a28e774513619c0c6259da,0x723bee2edd9595596a0ca2fa71ce505695735568,0x4fba94363621a3b86417d6d0b2819583263a9bc0,0x08e0a3e752d1338c0f02b986b9504f1009a962ca,0x85f15d0e945d96cf43a9156bc6ff63f8821b904a,0x6fb53031e2b3d7d4d6f7fad6d69952f87477bc82,0xcbc7d0ff51d37b60ba741bf566496bba53b5eea2,0x230ae3b25b46a5b6dc60a4eed173e1abd6c24133,0x391102852da9bf9c7644dce39864f3edb2f9a162,0xb48cb3742868bfefe0b29c1f298dae0573e1c79c,0x68a375bf740163db10ef7c2f705fbab37b5cc6ea,0x4f0a9445b9b2d5a2c27814cdd92cf6da0e42dff8,0x80897383d295d79079dfec48f4343f121dcb7c03,0x74ae32114e19ab369f0f24cf894ce06546436e23,0x9f327969a2aead18c7d0424c537a05c597080261,0xd41cef4f

In [24]:
for x in creator_addresses:
    if len(x) == 42:
        print(f"{x},")

0xd0966b2d8d48e057105afa7a1b84bf1188fb00f0,
0x72ba95d9701b0ae496ab4320010a43a764905b1b,
0x5de2b5b72a0b49f7d4490a1fef61afa8415971f5,
0xb1abf1039323f248514594a5ed1aef4f4fc95823,
0x4b4324bcc6db9380abbbbd20b24a16c11fb5b38a,
0x6826ba168776251688d4e3a6c4ac430cf4f185a8,
0x2ea49c863be976ee9c10cb8ef382757e48f0b9e5,
0x8f87d1feefa305a453174982df870fa93ebe2175,
0x21339e2ce5ce1d7a19a28e774513619c0c6259da,
0x723bee2edd9595596a0ca2fa71ce505695735568,
0x4fba94363621a3b86417d6d0b2819583263a9bc0,
0x08e0a3e752d1338c0f02b986b9504f1009a962ca,
0x85f15d0e945d96cf43a9156bc6ff63f8821b904a,
0x6fb53031e2b3d7d4d6f7fad6d69952f87477bc82,
0xcbc7d0ff51d37b60ba741bf566496bba53b5eea2,
0x230ae3b25b46a5b6dc60a4eed173e1abd6c24133,
0x391102852da9bf9c7644dce39864f3edb2f9a162,
0xb48cb3742868bfefe0b29c1f298dae0573e1c79c,
0x68a375bf740163db10ef7c2f705fbab37b5cc6ea,
0x4f0a9445b9b2d5a2c27814cdd92cf6da0e42dff8,
0x80897383d295d79079dfec48f4343f121dcb7c03,
0x74ae32114e19ab369f0f24cf894ce06546436e23,
0x9f327969a2aead18c7d0424c537a05

In [19]:
pd.Series([len(x) for x in creator_addresses]).sort_values()

276    41
0      42
201    42
200    42
199    42
       ..
97     42
96     42
95     42
93     42
298    42
Length: 299, dtype: int64

In [ ]:
"""

WITH nft_mints AS (
  SELECT 
    contract_address,
    "to" AS minter,
    tokenId,
    evt_block_time AS mint_time
  FROM erc721."ERC721_evt_Transfer"
  WHERE "from" = '\x0000000000000000000000000000000000000000'
    AND "to" IN ({{creator_addresses}})
)

SELECT 
  contract_address,
  COUNT(*) AS num_minted,
  MIN(mint_time) AS first_mint,
  MAX(mint_time) AS last_mint
FROM nft_mints
GROUP BY contract_address
ORDER BY num_minted DESC
"""